<a href="https://colab.research.google.com/github/taasinsaquib/retina/blob/main/onv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd 'drive/MyDrive/thesis_stuff/'

/content/drive/MyDrive/thesis_stuff


In [3]:
! pip install livelossplot torchinfo --quiet

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from   torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
from   torchvision import transforms, utils
from   torchinfo   import summary

import time
import copy
import sys

# specific package for visualization
from livelossplot import PlotLosses

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
seed = 28

# ONV has 14400 photoreceptors, reshape to a square for RESNET
w = 120
h = 120

# Data

## Combine and Shuffle Collected Data

In [12]:
sizes = [0.1, 0.2, 0.4, 0.6, 0.8, 1]
types = ['L', 'R']

In [13]:
data   = None
labels = None

for s in sizes:
  for t in types:

    d = np.load(f'./retina_LR/data_dist_{s}_{t}.npy')
    l = np.load(f'./retina_LR/labels_dist_{s}_{t}.npy')

    if data is None:
      data   = d
      labels = l
  
    else:
      data   = np.vstack((data, d))
      labels = np.vstack((labels, l))

In [14]:
shuffler = np.random.RandomState(seed=seed).permutation(len(labels))

data_shuffled   = data[shuffler]
labels_shuffled = labels[shuffler]

np.save('./retina_LR/data_combined',   data_shuffled)
np.save('./retina_LR/labels_combined', labels_shuffled)

## Load Combined Data

In [15]:
data   = np.load('./retina_LR/data_combined.npy')
labels = np.load('./retina_LR/labels_combined.npy')
print(data.shape, labels.shape)

(4080, 14400) (4080, 3)


##Create Torch Datasets (Loaders)

In [17]:
class ONVData(Dataset):

    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, index):

        x, y = self.subset[index]

        # ResNET 2 channels

        # TODO: make square 120x120
        x = np.resize(x, (w, h))

        posIdx = np.where(x == 1)
        negIdx = np.where(x == -1)

        channel1 = channel2 = np.zeros((w, h))
        newData = np.zeros((2, w, h), dtype=np.float)

        newData[0][posIdx] = 1
        newData[1][negIdx] = -1

        # print(type(x))
        x = torch.from_numpy(newData)
        # print(type(x))

        if self.transform:
          x = self.transform(x)

        return x, y

In [18]:
X_data_tensor = torch.from_numpy(data).float()
y_data_tensor = torch.from_numpy(labels).float()
init_dataset = TensorDataset(X_data_tensor, y_data_tensor)

In [19]:
# split train, val, test
lengths = np.array([0.7, 0.1, 0.2])
lengths *= int(len(init_dataset))
lengths = np.rint(lengths)
lengths = np.asarray(lengths, dtype=np.int32)

# lengths[1] += 1

In [20]:
print(lengths, np.sum(lengths), len(init_dataset))

[2856  408  816] 4080 4080


In [22]:
subset_train, subset_val, subset_test = random_split(init_dataset, lengths, generator=torch.Generator().manual_seed(seed)) 

train_data = ONVData(
    subset_train, transform=None)

val_data = ONVData(
    subset_val, transform=None)

test_data = ONVData(
    subset_test, transform=None)

dataloaders = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True,  num_workers=1),
    'val':   torch.utils.data.DataLoader(val_data,   batch_size=8,  shuffle=False, num_workers=1),
    'test':  torch.utils.data.DataLoader(test_data,  batch_size=8,  shuffle=False, num_workers=1),
}